<a href="https://colab.research.google.com/github/EmperoR1127/CSI5155_project/blob/master/csi5155_assignment_2_part_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
from scipy.io import arff
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "/content/drive/My Drive/Images/"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [0]:
#load four dataset
seismic_bumps_data = pd.DataFrame(arff.loadarff('/content/drive/My Drive/Data/seismic-bumps.arff')[0])
labor_neg_data = pd.read_csv("/content/drive/My Drive/Data/labor-neg.csv")
iris_data = pd.read_csv("/content/drive/My Drive/Data/iris.csv")
voting_data = pd.read_csv("/content/drive/My Drive/Data/voting-records.csv")

In [4]:
#process the seismic_bumps_data
#pre-process the train_set
sb_train_labels = seismic_bumps_data[["class"]].copy()
sb_train_set = seismic_bumps_data.drop(["class"], axis=1)
sb_train_labels["class"] = sb_train_labels["class"].map(lambda x: str(x)[2])
sb_train_set_num = seismic_bumps_data.drop(["seismic","seismoacoustic","shift", "ghazard", "class"], axis=1)
sb_train_set_cat = seismic_bumps_data.drop(["genergy","gpuls","gdenergy", "gdpuls", "nbumps", "nbumps2", "nbumps3", "nbumps4", "nbumps5", "nbumps6", "nbumps7", "nbumps89", "energy", "maxenergy", "class"], axis=1)
#build the pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
sb_num_pipeline = Pipeline([('imputer', SimpleImputer(strategy="median")),('std_scaler', StandardScaler()),])
sb_full_pipeline = ColumnTransformer([("num", sb_num_pipeline, list(sb_train_set_num)),("cat", OneHotEncoder(), list(sb_train_set_cat)),])
#prepare the data
sb_train_set_prepared = sb_full_pipeline.fit_transform(sb_train_set)
#prepare the target
sb_encoder = LabelEncoder()
sb_train_labels_prepared = sb_encoder.fit_transform(sb_train_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
#process the labor_neg_data
ln_train_labels = labor_neg_data['class'].copy()
ln_train_set = labor_neg_data.drop(["class"], axis = 1)
ln_train_set_num = ln_train_set.drop(["cola","pension","educ_allw", "vacation", "lngtrm_disabil", "dntl_ins", "bereavement", "empl_hplan"], axis=1)
ln_train_set_cat = ln_train_set.drop(["dur","wage1","wage2", "wage3", "hours", "stby_pay", "shift_diff", "holidays"], axis=1)
#build the pipeline
ln_num_pipeline = Pipeline([('imputer', SimpleImputer(strategy="median")),('std_scaler', StandardScaler()),])
ln_full_pipeline = ColumnTransformer([("num", ln_num_pipeline, list(ln_train_set_num)),("cat", OneHotEncoder(), list(ln_train_set_cat)),])
#prepare the data
ln_train_set_prepared = ln_full_pipeline.fit_transform(ln_train_set)
#prepare the target
ln_encoder = LabelEncoder()
ln_train_labels_prepared = ln_encoder.fit_transform(ln_train_labels)

In [0]:
#process the iris_data
ir_train_labels = iris_data['class'].copy()
ir_train_set = iris_data.drop(["class"], axis = 1)
#build the pipeline
ir_num_pipeline = Pipeline([('imputer', SimpleImputer(strategy="median")),('std_scaler', StandardScaler()),])
ir_full_pipeline = ColumnTransformer([("num", ir_num_pipeline, list(ir_train_set)),])
#prepare the data
ir_train_set_prepared = ir_full_pipeline.fit_transform(ir_train_set)
#prepare the target
ir_encoder = LabelEncoder()
ir_train_labels_prepared = ir_encoder.fit_transform(ir_train_labels)

In [0]:
#process the voting_data
vd_train_labels = voting_data['Class'].copy()
vd_train_set = voting_data.drop(["Class"], axis = 1)
#build the pipeline
vd_full_pipeline = ColumnTransformer([("cat", OneHotEncoder(), list(vd_train_set)),])
#prepare the data
vd_train_set_prepared = vd_full_pipeline.fit_transform(vd_train_set)
#prepare the target
vd_encoder = LabelEncoder()
vd_train_labels_prepared = vd_encoder.fit_transform(vd_train_labels)

In [18]:
#train the model using DECISION TREE algorithm with 10 fold cross validation
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
score_method = make_scorer(accuracy_score)
dt_clf = DecisionTreeClassifier()

dt_sb_train_prediction = cross_val_predict(dt_clf.fit(sb_train_set_prepared, sb_train_labels_prepared), sb_train_set_prepared, sb_train_labels_prepared, cv=10)
dt_sb_accuracy_score = accuracy_score(sb_train_labels_prepared, dt_sb_train_prediction)
print("The accuracies of the DECISION TREE algorithm trained against seismic_bumps_data is + %f " % dt_sb_accuracy_score)
print("-----------------------------------------")
dt_ln_train_prediction = cross_val_predict(dt_clf.fit(ln_train_set_prepared, ln_train_labels_prepared), ln_train_set_prepared, ln_train_labels_prepared, cv=10)
dt_ln_accuracy_score = accuracy_score(ln_train_labels_prepared, dt_ln_train_prediction)
print("The accuracies of the DECISION TREE algorithm trained against labor_neg_data is + %f " % dt_ln_accuracy_score)
print("-----------------------------------------")
dt_ir_train_prediction = cross_val_predict(dt_clf.fit(ir_train_set_prepared, ir_train_labels_prepared), ir_train_set_prepared, ir_train_labels_prepared, cv=10)
dt_ir_accuracy_score = accuracy_score(ir_train_labels_prepared, dt_ir_train_prediction)
print("The accuracies of the DECISION TREE algorithm trained against iris_data is + %f " % dt_ir_accuracy_score)
print("-----------------------------------------")
dt_vd_train_prediction = cross_val_predict(dt_clf.fit(vd_train_set_prepared, vd_train_labels_prepared), vd_train_set_prepared, vd_train_labels_prepared, cv=10)
dt_vd_accuracy_score = accuracy_score(vd_train_labels_prepared, dt_vd_train_prediction)
print("The accuracies of the DECISION TREE algorithm trained against voting_data is + %f " % dt_vd_accuracy_score)

The accuracies of the DECISION TREE algorithm trained against seismic_bumps_data is + 0.847910 
-----------------------------------------
The accuracies of the DECISION TREE algorithm trained against labor_neg_data is + 0.877193 
-----------------------------------------
The accuracies of the DECISION TREE algorithm trained against iris_data is + 0.953333 
-----------------------------------------
The accuracies of the DECISION TREE algorithm trained against voting_data is + 0.949425 


In [19]:
#train the model using K NEAREST NEIGHBOURS algorithm with 10 fold cross validation
from sklearn import neighbors
n_neighbors = 5
knn_clf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')

knn_sb_train_prediction = cross_val_predict(knn_clf.fit(sb_train_set_prepared, sb_train_labels_prepared), sb_train_set_prepared, sb_train_labels_prepared, cv=10)
knn_sb_accuracy_score = accuracy_score(sb_train_labels_prepared, knn_sb_train_prediction)
print("The accuracies of K NEAREST NEIGHBOURS algorithm trained against seismic_bumps_data is + %f " % knn_sb_accuracy_score)
print("-----------------------------------------")
knn_ln_train_prediction = cross_val_predict(knn_clf.fit(ln_train_set_prepared, ln_train_labels_prepared), ln_train_set_prepared, ln_train_labels_prepared, cv=10)
knn_ln_accuracy_score = accuracy_score(ln_train_labels_prepared, knn_ln_train_prediction)
print("The accuracies of K NEAREST NEIGHBOURS algorithm trained against labor_neg_data is + %f " % knn_ln_accuracy_score)
print("-----------------------------------------")
knn_ir_train_prediction = cross_val_predict(knn_clf.fit(ir_train_set_prepared, ir_train_labels_prepared), ir_train_set_prepared, ir_train_labels_prepared, cv=10)
knn_ir_accuracy_score = accuracy_score(ir_train_labels_prepared, knn_ir_train_prediction)
print("The accuracies of K NEAREST NEIGHBOURS algorithm trained against iris_data is + %f " % knn_ir_accuracy_score)
print("-----------------------------------------")
knn_vd_train_prediction = cross_val_predict(knn_clf.fit(vd_train_set_prepared, vd_train_labels_prepared), vd_train_set_prepared, vd_train_labels_prepared, cv=10)
knn_vd_accuracy_score = accuracy_score(vd_train_labels_prepared, knn_vd_train_prediction)
print("The accuracies of K NEAREST NEIGHBOURS algorithm trained against voting_data is + %f " % knn_vd_accuracy_score)

The accuracies of K NEAREST NEIGHBOURS algorithm trained against seismic_bumps_data is + 0.908282 
-----------------------------------------
The accuracies of K NEAREST NEIGHBOURS algorithm trained against labor_neg_data is + 0.947368 
-----------------------------------------
The accuracies of K NEAREST NEIGHBOURS algorithm trained against iris_data is + 0.953333 
-----------------------------------------
The accuracies of K NEAREST NEIGHBOURS algorithm trained against voting_data is + 0.921839 


In [21]:
#train the model using NAIVE BAYES algorithm with 10 fold cross validation
from sklearn.naive_bayes import GaussianNB
gnb_clf = GaussianNB()

gnb_sb_train_prediction = cross_val_predict(gnb_clf.fit(sb_train_set_prepared, sb_train_labels_prepared), sb_train_set_prepared, sb_train_labels_prepared, cv=10)
gnb_sb_accuracy_score = accuracy_score(sb_train_labels_prepared, gnb_sb_train_prediction)
print("The accuracies of NAIVE BAYES algorithm trained against seismic_bumps_data is + %f " % gnb_sb_accuracy_score)
print("-----------------------------------------")
gnb_ln_train_prediction = cross_val_predict(gnb_clf.fit(ln_train_set_prepared, ln_train_labels_prepared), ln_train_set_prepared, ln_train_labels_prepared, cv=10)
gnb_ln_accuracy_score = accuracy_score(ln_train_labels_prepared, gnb_ln_train_prediction)
print("The accuracies of NAIVE BAYES algorithm trained against labor_neg_data is + %f " % gnb_ln_accuracy_score)
print("-----------------------------------------")
gnb_ir_train_prediction = cross_val_predict(gnb_clf.fit(ir_train_set_prepared, ir_train_labels_prepared), ir_train_set_prepared, ir_train_labels_prepared, cv=10)
gnb_ir_accuracy_score = accuracy_score(ir_train_labels_prepared, gnb_ir_train_prediction)
print("The accuracies of NAIVE BAYES algorithm trained against iris_data is + %f " % gnb_ir_accuracy_score)
print("-----------------------------------------")
gnb_vd_train_prediction = cross_val_predict(gnb_clf.fit(vd_train_set_prepared, vd_train_labels_prepared), vd_train_set_prepared, vd_train_labels_prepared, cv=10)
gnb_vd_accuracy_score = accuracy_score(vd_train_labels_prepared, gnb_vd_train_prediction)
print("The accuracies of NAIVE BAYES algorithm trained against voting_data is + %f " % gnb_vd_accuracy_score)

The accuracies of NAIVE BAYES algorithm trained against seismic_bumps_data is + 0.382740 
-----------------------------------------
The accuracies of NAIVE BAYES algorithm trained against labor_neg_data is + 0.929825 
-----------------------------------------
The accuracies of NAIVE BAYES algorithm trained against iris_data is + 0.953333 
-----------------------------------------
The accuracies of NAIVE BAYES algorithm trained against voting_data is + 0.944828 


In [22]:
#train the model using RULE BASED algorithm with 10 fold cross validation
from sklearn.dummy import DummyClassifier
dc_clf = DummyClassifier(strategy='stratified')

dc_sb_train_prediction = cross_val_predict(dc_clf.fit(sb_train_set_prepared, sb_train_labels_prepared), sb_train_set_prepared, sb_train_labels_prepared, cv=10)
dc_sb_accuracy_score = accuracy_score(sb_train_labels_prepared, dc_sb_train_prediction)
print("The accuracies of RULE BASED algorithm trained against seismic_bumps_data is + %f " % dc_sb_accuracy_score)
print("-----------------------------------------")
dc_ln_train_prediction = cross_val_predict(dc_clf.fit(ln_train_set_prepared, ln_train_labels_prepared), ln_train_set_prepared, ln_train_labels_prepared, cv=10)
dc_ln_accuracy_score = accuracy_score(ln_train_labels_prepared, dc_ln_train_prediction)
print("The accuracies of RULE BASED algorithm trained against labor_neg_data is + %f " % dc_ln_accuracy_score)
print("-----------------------------------------")
dc_ir_train_prediction = cross_val_predict(dc_clf.fit(ir_train_set_prepared, ir_train_labels_prepared), ir_train_set_prepared, ir_train_labels_prepared, cv=10)
dc_ir_accuracy_score = accuracy_score(ir_train_labels_prepared, dc_ir_train_prediction)
print("The accuracies of RULE BASED algorithm trained against iris_data is + %f " % dc_ir_accuracy_score)
print("-----------------------------------------")
dc_vd_train_prediction = cross_val_predict(dc_clf.fit(vd_train_set_prepared, vd_train_labels_prepared), vd_train_set_prepared, vd_train_labels_prepared, cv=10)
dc_vd_accuracy_score = accuracy_score(vd_train_labels_prepared, dc_vd_train_prediction)
print("The accuracies of RULE BASED algorithm trained against voting_data is + %f " % dc_vd_accuracy_score)

The accuracies of RULE BASED algorithm trained against seismic_bumps_data is + 0.876548 
-----------------------------------------
The accuracies of RULE BASED algorithm trained against labor_neg_data is + 0.543860 
-----------------------------------------
The accuracies of RULE BASED algorithm trained against iris_data is + 0.280000 
-----------------------------------------
The accuracies of RULE BASED algorithm trained against voting_data is + 0.494253 


In [26]:
import scikit_posthocs as sp
x = [[3,1,4,2],[3,1,2,4],[1,1,1,4],[1,3,2,4]]
sp.posthoc_nemenyi(x)

,1,2,3,4
1,-1.000000,1.000000,0.834769,1.000000
2,1.000000,-1.000000,0.834769,1.000000
3,0.834769,0.834769,-1.000000,0.834769
4,1.000000,1.000000,0.834769,-1.000000


In [27]:
sp.posthoc_nemenyi_friedman(x)

,0,1,2,3
0,-1.000000,0.900000,0.900000,0.516551
1,0.900000,-1.000000,0.823993,0.220908
2,0.900000,0.823993,-1.000000,0.670273
3,0.516551,0.220908,0.670273,-1.000000
